In [1]:
from data.tinyImageNet import tinyImageNetVague 

# data_dir = '/data/cxl173430/HyperEvidentialNN/TinyImageNet/'
data_dir = '/home/cxl173430/Documents/projects/uncertainty_Related/HENN_originalNotebooks/'
batch_size = 64

mydata = tinyImageNetVague(
            data_dir, 
            num_comp=1, 
            batch_size=batch_size,
            imagenet_hierarchy_path=data_dir,
            duplicate=True)

Loading TinyImageNet...
length of parent_to_subclass:  107
Total 29 Candidate superclasses: ['n04371563', 'n03419014', 'n04490091', 'n02924116', 'n02858304', 'n04379243', 'n07557434', 'n07611358', 'n07707451', 'n03748162', 'n03259505', 'n02898711', 'n03743902', 'n02796623', 'n09433442', 'n03278248', 'n04438304', 'n04147495', 'n04565375', 'n03880531', 'n02876657', 'n01767661', 'n01772222', 'n01942177', 'n02470325', 'n02370806', 'n02075296', 'n02121808', 'n01844917']
Vague classes: ['n03419014']
Vague classes nid: [['n02730930', 'n03763968']]
Vague classes ids: [[62, 119]]
Actual label sets
 R: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [

In [2]:
print(len(mydata.train_loader.dataset)) # >90200 because of the duplicates
print(len(mydata.valid_loader.dataset))
print(len(mydata.test_loader.dataset))

90498
9800
10000


## duplicate validation set

In [3]:
# Import libraries
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
import sys 
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from random import randint
from datetime import datetime as dt
import time

from sklearn import metrics

import math
import torch
from torch import optim, nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.utils.data import Subset, ConcatDataset
from torch.utils.data import random_split, sampler 
from torchvision.utils import make_grid
from torchvision import transforms as T
from torchvision import models, datasets, utils 
# from efficientnet_pytorch import EfficientNet

In [4]:
class ReduceLabelDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        x, y_truth_single, y = self.dataset[index]
        return x, y
        
    def __len__(self):
        return len(self.dataset)
    
valid_ds = mydata.modify_vague_samples(mydata.valid_loader.dataset)
valid_ds_reducedLabel = ReduceLabelDataset(valid_ds)
valid_loader = DataLoader(valid_ds_reducedLabel, batch_size=64, num_workers=1, pin_memory=True)

In [5]:
mydata.valid_loader.dataset.dataset

# Load Pretrained Traditional CNN based on duplicates

In [6]:
import copy
from backbones import EfficientNet_pretrain
num_singles = 200
model = EfficientNet_pretrain(num_singles)

Loaded pretrained weights for efficientnet-b3


In [7]:
# output_folder="/data/cxl173430/uncertainty_Related/HENN_Git_VScode/HyperEvidentialNN_Results/"
output_folder="/home/cxl173430/Documents/projects/uncertainty_Related/HENN_Git_VScode/HyperEvidentialNN_Results/"
saved_spec_dir = "tiny_baseline_DNN"
base_path = os.path.join(output_folder, saved_spec_dir)
base_path

'/home/cxl173430/Documents/projects/uncertainty_Related/HENN_Git_VScode/HyperEvidentialNN_Results/tiny_baseline_DNN'

In [35]:
# valid_loader = mydata.valid_loader
test_loader = mydata.test_loader
R = mydata.R
saved_path = os.path.join(base_path, "model_CrossEntropy.pt")
checkpoint = torch.load(saved_path, map_location="cuda:0")
model.load_state_dict(checkpoint["model_state_dict"])

model_best_from_valid = copy.deepcopy(model)
model_best_from_valid.load_state_dict(checkpoint["model_state_dict_best"]) 

# # model after the final epoch
# if args.vaguePred:
#     js_result, prec_recall_f = evaluate_vague_final(model, test_loader, valid_loader, R, device)
# if args.nonVaguePred:
#     acc_nonvague = evaluate_nonvague_final(model, test_loader, device)
# test_result_log(js_result, prec_recall_f, acc_nonvague, False) # bestModel=False

# print(f"### Use the model selected from validation set in Epoch {checkpoint['epoch_best']}:\n")
# if args.vaguePred:
#     js_result, prec_recall_f = evaluate_vague_final(model_best_from_valid, test_loader, valid_loader, R, device)
# if args.nonVaguePred:
#     acc_nonvague = evaluate_nonvague_final(model_best_from_valid, test_loader, device)
# test_result_log(js_result, prec_recall_f, acc_nonvague, True)

<All keys matched successfully>

# **RAPS**

In [36]:
import os, sys, inspect
# raps_path = "/data/cxl173430/HyperEvidentialNN/Baseline-RAPS"
raps_path = "/home/cxl173430/Documents/projects/uncertainty_Related/HENN_Git_VScode/Baseline_RAPS/"
sys.path.insert(1, os.path.join(raps_path, './conformal_classification/'))
# sys.path.insert(1, os.path.join(sys.path[0], './conformal_classification/'))

from conformal import *
from utils import *

# Import other standard packages

import torchvision

import matplotlib.pyplot as plt 

import torch.backends.cudnn as cudnn


# Fix the random seed for reproducibility (you can change this, of course) 
seed=0
np.random.seed(seed=seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)

# # Normalization from torchvision repo
# transform = T.Compose([
#                 T.Resize(256),
#                 T.CenterCrop(224),
#                 T.ToTensor(),
#                 T.Normalize(mean=[0.485, 0.456, 0.406],
#                                       std= [0.229, 0.224, 0.225])
#             ])

cudnn.benchmark = True
batch_size = 128

In [48]:
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import Counter
def precision_recall_f_v1(y_test, y_pred):
    # make singleton labels 0, and composite labels 1
    y_test = y_test.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    label_value_cnt = Counter(y_test)
    pred_value_cnt = Counter(y_pred)

    comp_GT_cnt = label_value_cnt[True]
    cmp_pred_cnt = pred_value_cnt[True]
    
    return precision, recall, f1, comp_GT_cnt, cmp_pred_cnt

# Model after final epoch

In [49]:
# Get your model
_ = model.eval()
model.to("cuda:0")

EfficientNet_pretrain(
  (network): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          40, 10, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          10, 40, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStati

In [50]:
# Get the conformal calibcration dataset

# Initialize loaders 
calib_loader = valid_loader

# Conformalize model
cmodel = ConformalModel(model, calib_loader, alpha=0.1, lamda_criterion='size')

Begin Platt scaling.
Computing logits for model (only happens once).


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 154/154 [01:13<00:00,  2.10it/s]


Optimal T=1.3972055912017822


In [51]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [52]:
correct_vague = 0.0
correct_nonvague = 0.0
vague_total = 0
nonvague_total = 0

test_ds = mydata.test_loader.dataset

#vague prediction
pred_sets = []
ground_sets = []

#nonvague prediction
num_corr_1 = 0 # use original label before blurring
num_corr_2 = 0 # if nonvague prediction is in composite labels, then +1
num_corr_3 = 0 # similiar to JS

comp_or_not_original = []
comp_or_not_pred = []

for data in test_ds:
    img, label_singl, label = data
    scores, set_pred_array = cmodel(img.view(1,3,224,224).cuda())
    set_predict = {s for s in set_pred_array[0]}
    set_ground = {s for s in R[label]}
    
    pred_sets.append(set_predict)
    ground_sets.append(set_ground)
 
    inter_set = set_predict.intersection(set_ground)
    union_set = set_predict.union(set_ground)
#     print("\n")
    
#     print("preded   set: ", set_predict)
#     print("ground truth: ", set_ground)
#     print("inter    set: ", inter_set)
#     print("union    set: ", union_set)

    rate = float(len(inter_set)/len(union_set))
    if len(set_ground) == 1:
        correct_nonvague += rate
        nonvague_total += 1
        comp_or_not_original.append(0)
    else:
        correct_vague += rate 
        vague_total += 1
        comp_or_not_original.append(1)
    
    if len(set_predict) == 1:
        comp_or_not_pred.append(0)
    else:
        comp_or_not_pred.append(1)
        
    ### nonvague prediction
    pred_singl = torch.argmax(scores, dim=1).cpu().item()
    # 1.
    if pred_singl == label_singl:
        num_corr_1 += 1
    # 2.
    if pred_singl in set_ground:
        num_corr_2 += 1
    # 3.
    inter_tmp = set_ground.intersection(set([pred_singl]))
    union_tmp = set_ground.union(set([pred_singl]))
    num_corr_3 += float(len(inter_tmp)/len(union_tmp))
    ### END
    
#     break
print(pred_singl, label_singl, num_corr_1)
print(pred_singl, set_ground, num_corr_2)
print(pred_singl, set_ground, num_corr_3)

comp_or_not_original = torch.tensor(comp_or_not_original)
comp_or_not_pred = torch.tensor(comp_or_not_pred)
print("### precision, recall, f1, comp_GT_cnt, cmp_pred_cnt:")
print(precision_recall_f_v1(comp_or_not_original, comp_or_not_pred))

stat_result=[correct_nonvague, correct_vague, nonvague_total, vague_total]

avg_js_nonvague = stat_result[0] / (stat_result[2]+1e-10)
avg_js_vague = stat_result[1] / (stat_result[3]+1e-10)
overall_js = (stat_result[0] + stat_result[1])/(stat_result[2] + stat_result[3]+1e-10)

print(stat_result)
print(f"Overall JS: {overall_js}")
print(f"Comp JS Vague Classes:{avg_js_vague}")
print(f"Singleton JS Nonvague Classes:{avg_js_nonvague}")


acc_1 = num_corr_1 / len(test_ds)
acc_2 = num_corr_2 / len(test_ds)
acc_3 = num_corr_3 / len(test_ds)
print(f"acc 1: {acc_1:.4f}, acc 2: {acc_2:.4f}, acc3: {acc_3:.4f}")

119 119 7978
119 {62, 119} 7995
119 {62, 119} 7979.0
### precision, recall, f1, comp_GT_cnt, cmp_pred_cnt:
(0.0, 0.0, 0.0, 32, 362)
[7956.0, 16.0, 9968, 32]
Overall JS: 0.797199999999992
Comp JS Vague Classes:0.49999999999843747
Singleton JS Nonvague Classes:0.7981540930979053
acc 1: 0.7978, acc 2: 0.7995, acc3: 0.7979


# best model

In [53]:
# Get your model
_ = model_best_from_valid.eval()
model_best_from_valid.to("cuda:0")

# Get the conformal calibcration dataset

# Initialize loaders 
calib_loader = valid_loader

# Conformalize model
cmodel = ConformalModel(model_best_from_valid, calib_loader, alpha=0.1, lamda_criterion='size')

Begin Platt scaling.
Computing logits for model (only happens once).


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 154/154 [01:13<00:00,  2.08it/s]


Optimal T=0.8852202296257019


In [54]:
correct_vague = 0.0
correct_nonvague = 0.0
vague_total = 0
nonvague_total = 0

test_ds = mydata.test_loader.dataset

#vague prediction
pred_sets = []
ground_sets = []

#nonvague prediction
num_corr_1 = 0 # use original label before blurring
num_corr_2 = 0 # if nonvague prediction is in composite labels, then +1
num_corr_3 = 0 # similiar to JS

comp_or_not_original = []
comp_or_not_pred = []

for data in test_ds:
    img, label_singl, label = data
    scores, set_pred_array = cmodel(img.view(1,3,224,224).cuda())
    set_predict = {s for s in set_pred_array[0]}
    set_ground = {s for s in R[label]}
    
    pred_sets.append(set_predict)
    ground_sets.append(set_ground)
 
    inter_set = set_predict.intersection(set_ground)
    union_set = set_predict.union(set_ground)
#     print("\n")
    
#     print("preded   set: ", set_predict)
#     print("ground truth: ", set_ground)
#     print("inter    set: ", inter_set)
#     print("union    set: ", union_set)

    rate = float(len(inter_set)/len(union_set))
    if len(set_ground) == 1:
        correct_nonvague += rate
        nonvague_total += 1
        comp_or_not_original.append(0)
    else:
        correct_vague += rate 
        vague_total += 1
        comp_or_not_original.append(1)
    
    if len(set_predict) == 1:
        comp_or_not_pred.append(0)
    else:
        comp_or_not_pred.append(1)
        
    ### nonvague prediction
    pred_singl = torch.argmax(scores, dim=1).cpu().item()
    # 1.
    if pred_singl == label_singl:
        num_corr_1 += 1
    # 2.
    if pred_singl in set_ground:
        num_corr_2 += 1
    # 3.
    inter_tmp = set_ground.intersection(set([pred_singl]))
    union_tmp = set_ground.union(set([pred_singl]))
    num_corr_3 += float(len(inter_tmp)/len(union_tmp))
    ### END
    
#     break
print(pred_singl, label_singl, num_corr_1)
print(pred_singl, set_ground, num_corr_2)
print(pred_singl, set_ground, num_corr_3)

comp_or_not_original = torch.tensor(comp_or_not_original)
comp_or_not_pred = torch.tensor(comp_or_not_pred)
print("### precision, recall, f1, comp_GT_cnt, cmp_pred_cnt:")
print(precision_recall_f_v1(comp_or_not_original, comp_or_not_pred))

stat_result=[correct_nonvague, correct_vague, nonvague_total, vague_total]

avg_js_nonvague = stat_result[0] / (stat_result[2]+1e-10)
avg_js_vague = stat_result[1] / (stat_result[3]+1e-10)
overall_js = (stat_result[0] + stat_result[1])/(stat_result[2] + stat_result[3]+1e-10)

print(stat_result)
print(f"Overall JS: {overall_js}")
print(f"Comp JS Vague Classes:{avg_js_vague}")
print(f"Singleton JS Nonvague Classes:{avg_js_nonvague}")


acc_1 = num_corr_1 / len(test_ds)
acc_2 = num_corr_2 / len(test_ds)
acc_3 = num_corr_3 / len(test_ds)
print(f"acc 1: {acc_1:.4f}, acc 2: {acc_2:.4f}, acc3: {acc_3:.4f}")

119 119 8220
119 {62, 119} 8236
119 {62, 119} 8220.0
### precision, recall, f1, comp_GT_cnt, cmp_pred_cnt:
(0.0, 0.0, 0.0, 32, 253)
[8201.5, 16.0, 9968, 32]
Overall JS: 0.8217499999999918
Comp JS Vague Classes:0.49999999999843747
Singleton JS Nonvague Classes:0.822782905296942
acc 1: 0.8220, acc 2: 0.8236, acc3: 0.8220


In [32]:
img.shape

torch.Size([3, 224, 224])